In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import pandas as pd
import json

In [5]:
df = pd.read_json("/content/drive/MyDrive/NLP/EvolInstruct-Code-80k.json")

In [6]:
df

,instruction,output
0,Create a nested loop to print every combinatio...,Here is an example of a nested loop in Python ...
1,Increase the difficulty by asking to print the...,Here's the updated code to print the square of...
2,Write a function to find the number of distinc...,The given problem can be solved by iterating t...
3,Write code that removes spaces and punctuation...,Here's an example of code that attempts to sol...
4,Write a function that checks if a given number...,Here is an implementation of the function in P...
...,...,...
78259,Identify and display all the data types within...,SELECT DISTINCT data_type \nFROM products \nWH...
78260,Create a basic Python script to accept two num...,"num1 = int(input(""Enter the first number: ""))\..."
78261,Create a basic C# program to print out the cur...,using System;\n\npublic class Program \n{\n ...
78262,Find and display all the employees who earn mo...,SELECT e.name \nFROM employees e \nWHERE e.sal...


#### Create Formatted Prompt

In the following function we'll be merging our `prompt` and `response` columns by creating the following template:

```
<s>### Instruction:
Use the provided input to create an instruction that could have been used to generate the response with an LLM.

### Input:
{input}

### Response:
{response}</s>
```

In [7]:
def create_prompt(sample):
  """
  Update the prompt template:
  Combine both the prompt and input into a single column.

  """
  bos_token = "<s>"
  #original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
  response = sample["output"]#.replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
  input = sample["instruction"]
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += "### Instruction:"
  full_prompt += "\n" + system_message
  full_prompt += "\n\n### Input:"
  full_prompt += "\n" + input
  full_prompt += "\n\n### Response:"
  full_prompt += "\n" + response
  full_prompt += eos_token

  return full_prompt

In [8]:
from datasets import Dataset, DatasetDict

In [9]:
train_size = int(len(df) * 0.8)  # 80% of the data for training
train_data = df[:train_size]
test_data = df[train_size:]

In [10]:
num_rows_train = len(train_data)
num_rows_test = len(test_data)

In [11]:
features = {
    'instruction': 'string',
    'output': 'string'
}

In [12]:
features = {
    'instruction': 'string',
    'output': 'string'
}

# Create DatasetDict with train and test subsets
dataset_dict = DatasetDict({
    'train': Dataset.from_pandas(train_data),
    'test': Dataset.from_pandas(test_data)
})

# Set features for each subset
dataset_dict = dataset_dict.map(lambda example: {'instruction': example['instruction'], 'output': example['output']}, batched=True)

# Display DatasetDict with train and test subsets
print(dataset_dict)

Map:   0%|          | 0/62611 [00:00<?, ? examples/s]

Map:   0%|          | 0/15653 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 62611
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 15653
    })
})


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [16]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [17]:
prompt="### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.\n\n### Response:"



In [18]:
generate_response(prompt, model)

'<s> \nLLM, when generating text about different types of grass, consider including information such as their name, growth rate, appearance, and soil preference. Provide examples of the most common and popular grass species, such as Kentucky Bluegrass and Bermuda grass. Additionally, consider mentioning any unique characteristics of each species, such as the shine and color of Rygrass and Fescues.</s>'

In [19]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [20]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [21]:
import torch

# Clear GPU cache
torch.cuda.empty_cache()

In [22]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_instruct_generation",
  #num_train_epochs=5,
  max_steps = 100, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 4,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
)

In [23]:
dataset_dict['test'][0]

{'instruction': "Write a JavaScript program to classify a given array of strings into two groups- one which contains elements that can be parsed into numbers and one which can't.",
 'output': 'function classify(arr) {\n  let nums = [];\n  let strs = [];\n  arr.forEach(el => {\n    if (!isNaN(el)) nums.push(el);\n    else strs.push(el);\n  });\n  return [nums, strs];\n}'}

In [24]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=args,
  train_dataset=dataset_dict["train"],
  eval_dataset=dataset_dict["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:342: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [25]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
20,0.558200,0.624736
40,0.497400,0.578219
60,0.499200,0.577507
80,0.491000,0.568454
100,0.478500,0.564044


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=100, training_loss=0.5123496723175048, metrics={'train_runtime': 6436.9375, 'train_samples_per_second': 0.062, 'train_steps_per_second': 0.016, 'total_flos': 3.50843194834944e+16, 'train_loss': 0.5123496723175048, 'epoch': 0.024218939210462583})

In [33]:
trainer.save_model("/content/drive/MyDrive/NLP/model/mistral_instruct_generation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub("yeshwanthkesani/mistral-instruct-generation")

events.out.tfevents.1714373039.1c3689a4a868.295.0:   0%|          | 0.00/5.20k [00:00<?, ?B/s]

events.out.tfevents.1714373436.1c3689a4a868.295.1:   0%|          | 0.00/5.20k [00:00<?, ?B/s]

events.out.tfevents.1714373594.1c3689a4a868.295.2:   0%|          | 0.00/8.94k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yeshwanthkesani/mistral_instruct_generation/commit/9244fd4184b3d818aa2513cfc82b02900cd5741d', commit_message='yeshwanthkesani/mistral-instruct-generation', commit_description='', oid='9244fd4184b3d818aa2513cfc82b02900cd5741d', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [ ]:
generate_response("### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:\nGive me bubble sort code in python.\n\n### Response:", merged_model)

'<s> ### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:\nGive me bubble sort code in python.\n\n### Response:\nHere is an implementation of bubble sort in Python:\n\n```python\ndef bubble_sort(arr):\n    # Get the length of the array\n    n = len(arr)\n    \n    # Loop through each element in the array\n    for i in range(n):\n        \n        # Last i elements are already sorted, so inner loop will go until n-i-1\n        for j in range(0, n - i - 1):\n            \n            # Swap if current element is greater than next one\n            if arr[j] > arr[j + 1]:\n                arr[j], arr[j + 1] = arr[j + 1], arr[j]\n                \n    # Return the sorted array\n    return arr\n```\nYou can use this function to sort an array of integers in Python. For example:\n\n```python\narr = [64, 34, 25, 12, 22, 11, 90]\nsorted_arr = bubble_sort(arr)\nprint(sorted_arr)\n```\n\nThis will output:\n```py

In [29]:
df

,instruction,output
0,Create a nested loop to print every combinatio...,Here is an example of a nested loop in Python ...
1,Increase the difficulty by asking to print the...,Here's the updated code to print the square of...
2,Write a function to find the number of distinc...,The given problem can be solved by iterating t...
3,Write code that removes spaces and punctuation...,Here's an example of code that attempts to sol...
4,Write a function that checks if a given number...,Here is an implementation of the function in P...
...,...,...
78259,Identify and display all the data types within...,SELECT DISTINCT data_type \nFROM products \nWH...
78260,Create a basic Python script to accept two num...,"num1 = int(input(""Enter the first number: ""))\..."
78261,Create a basic C# program to print out the cur...,using System;\n\npublic class Program \n{\n ...
78262,Find and display all the employees who earn mo...,SELECT e.name \nFROM employees e \nWHERE e.sal...


In [30]:
random_rows = df.sample(n=100, random_state=42)

In [32]:
random_rows

,instruction,output
34962,You are given a string array. Code a JavaScrip...,Here is a possible solution to remove duplicat...
7055,Write a Python function that takes in a list o...,Here is a Python function that satisfies the g...
39881,Find the length of a given string using JavaSc...,Here's an alternative approach to find the len...
64573,Create a Python dictionary with key-values pai...,"fruit_dict = {\n ""apple"":5,\n ""banana"":1..."
71808,What is the maximum length of a string in Python?,The maximum length of a string in Python is sy...
...,...,...
43511,You need to write a CSS rule to change the bac...,"To achieve this, you can use the `:nth-child` ..."
11790,Write a code that generates a random password ...,import random\nimport string\n\ndef generate_p...
6175,Write a multithreaded class in Java to submit ...,Here is an example of a multithreaded class in...
61284,Which of the following type is used to store c...,The data type used to store characters in memo...


In [37]:
random_rows['generated_response'] = random_rows['instruction'].apply(lambda x: generate_response(f"### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:\n{x}\n\n### Response:", model))

KeyboardInterrupt: 

In [38]:
generated_responses = []
from tqdm import tqdm

# Use tqdm with apply to show progress bar
for instruction in tqdm(random_rows['instruction']):
    input_text = f"### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:\n{instruction}\n\n### Response:"
    generated_response = generate_response(input_text, model)
    generated_responses.append(generated_response)

# Assign the generated responses to the DataFrame
random_rows['generated_response'] = generated_responses

100%|██████████| 100/100 [40:21<00:00, 24.22s/it]


In [39]:
csv_file_path = "/content/mistral_instruct_generation/generated_responses.csv"
random_rows.to_csv(csv_file_path, index=False)
random_rows

,instruction,output,generated_response
34962,You are given a string array. Code a JavaScrip...,Here is a possible solution to remove duplicat...,<s> \nWrite a JavaScript function to remove du...
7055,Write a Python function that takes in a list o...,Here is a Python function that satisfies the g...,<s> \nHere is a Python function that satisfies...
39881,Find the length of a given string using JavaSc...,Here's an alternative approach to find the len...,<s> \nCreate an instruction that asks for a Ja...
64573,Create a Python dictionary with key-values pai...,"fruit_dict = {\n ""apple"":5,\n ""banana"":1...",<s> \nHere's an example Python code that creat...
71808,What is the maximum length of a string in Python?,The maximum length of a string in Python is sy...,"<s> \nIn Python, there is no maximum length of..."
...,...,...,...
43511,You need to write a CSS rule to change the bac...,"To achieve this, you can use the `:nth-child` ...",<s> \nHere is a CSS rule that satisfies the gi...
11790,Write a code that generates a random password ...,import random\nimport string\n\ndef generate_p...,<s> \nTo generate a random password of length ...
6175,Write a multithreaded class in Java to submit ...,Here is an example of a multithreaded class in...,<s> \nTo create a multithreaded class in Java ...
61284,Which of the following type is used to store c...,The data type used to store characters in memo...,<s> \nThe type used to store characters in mem...


In [40]:
generated_responses_list = random_rows['generated_response'].tolist()
output_list = random_rows['output'].tolist()


In [45]:
from rouge_score import rouge_scorer
from collections import defaultdict
import numpy as np

# Define a function to compute average ROUGE scores
def compute_average_rouge_scores(generated_responses, reference_responses):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    aggregate_scores = defaultdict(list)

    # Compute ROUGE scores for each pair of generated and reference responses
    for generated, reference in zip(generated_responses, reference_responses):
        scores = scorer.score(reference, generated)
        for key, score in scores.items():
            aggregate_scores[key].append(score.fmeasure)  # Focusing on F-measure for average

    # Calculate the average score for each ROUGE metric
    average_scores = {key: np.mean(values) for key, values in aggregate_scores.items()}
    return average_scores

# Assuming `generated_responses` and `reference_responses` are lists of strings
# For example:
# generated_responses = ["generated sentence 1", "generated sentence 2", ...]
# reference_responses = ["reference sentence 1", "reference sentence 2", ...]

# Compute average ROUGE scores
average_rouge_scores = compute_average_rouge_scores(generated_responses_list, output_list)

# Print average ROUGE scores
print("Average ROUGE scores:")
for metric, score in average_rouge_scores.items():
    print(f"{metric}: {score:.2f}")


Average ROUGE scores:
rouge1: 0.42
rouge2: 0.22
rougeL: 0.29


In [48]:
# Other imports remain the same
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

# Batch processing function
def encode_responses_in_batches(model, responses, batch_size=64):
    embeddings = []
    for i in range(0, len(responses), batch_size):
        batch = responses[i:i+batch_size]
        batch_embeddings = model.encode(batch, convert_to_tensor=True, device='cpu')
        embeddings.append(batch_embeddings)
    embeddings = torch.cat(embeddings)  # Combine the batched embeddings
    return embeddings

# Updated function for computing average cosine similarity using batch processing
def compute_average_cosine_similarity(generated_responses, reference_responses, batch_size=64):
    # Load the model and set to CPU
    model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
    # Batch processing
    generated_embeddings = encode_responses_in_batches(model, generated_responses, batch_size)
    reference_embeddings = encode_responses_in_batches(model, reference_responses, batch_size)
    # Compute cosine similarities for each pair
    similarities = cosine_similarity(generated_embeddings, reference_embeddings)
    # Average similarity
    average_similarity = np.mean(np.diag(similarities))
    return average_similarity

# Assuming `generated_responses` and `reference_responses` are lists of strings
# Compute average cosine similarity
average_similarity = compute_average_cosine_similarity(generated_responses_list, output_list, batch_size=64)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [49]:
# Print average cosine similarity
print(f"Average Cosine Similarity: {average_similarity:.2f}")

Average Cosine Similarity: 0.77
